# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [ ]:
!pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math
import random

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.5 MB/s eta 0:00:00


**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

1. Haszowanie zapewnia jedynie integralność, ponieważ proces ten polega na przekształceniu wiadomości na wartość skrótu, który można później porównać z wartością skrótu oryginalnej wiadomości.

2. Kod MAC zapewnia poufność i integralność, ale nie zapewnia non-repudiation. Odbiorca wiadomości lub atakujący może nadal przesłać tę samą wiadomość, ponieważ kod MAC nie uwzględnia, kto i do czego go użył. Nawet popularny kod HMAC, który wymaga użycia klucza prywatnego, nie zapewnia non-repudiation, ponieważ klucz prywatny może być udostępniony wielu osobom, a HMAC nie umożliwia identyfikacji konkretnego użytkownika.

3. Podpis cyfrowy zapewnia wszystkie trzy cechy, czyli poufność, integralność i non-repudiation. Podpis cyfrowy wykorzystuje klucz prywatny do podpisywania wiadomości, a klucz publiczny do weryfikacji podpisu. Dzięki temu procesowi można jednoznacznie zidentyfikować nadawcę wiadomości i potwierdzić, że wiadomość nie została zmieniona od momentu podpisania.

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimpoertowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [ ]:
nlen = 2048

def gen_p_q():
  p, q = cu.getPrime(nlen // 2), cu.getPrime(nlen // 2)
  while not (abs(p - q) > pow(2, (nlen // 2) - 100)):
    p, q = cu.getPrime(nlen // 2), cu.getPrime(nlen // 2)

  return p, q

def gen_f_e(p, q):
  e = cu.getRandomRange(65537, pow(2, 256))
  # e = 65537
  f = (p - 1) * (q - 1)
  while not math.gcd(f, e):
    e = cu.getRandomRange(65537, pow(2, 256))

  return f, e

def gen_numbers_set():
  p, q = gen_p_q()
  f, e = gen_f_e(p, q)

  while math.lcm(p - 1, q - 1) < e * pow(2, nlen // 2):
    p, q = gen_p_q()
    f, e = gen_f_e(p, q)

  return p, q, f, e

def gen_private_key(p, q, f, e):
  n = p * q
  d = cu.inverse(e, f)

  return (d, n)

def gen_public_key(p, q, e):
  n = p * q

  return (e, n)

In [ ]:
gen_numbers_set_success = False
err_count = 0

while not gen_numbers_set_success:
  try:
    p, q, f, e = gen_numbers_set()

    # print(f'f=\n{f}')
    # print(f'e=\n{e}')

    private_key = gen_private_key(p, q, f, e)
    public_key = gen_public_key(p, q, e)
    gen_numbers_set_success = True
  except ValueError:
    err_count += 1
    print(f'{err_count} ValueError: base is not invertible for the given modules')

# Czesto wywala blad - musi byc jakis scenariusz do pokrycia z f i e - odpalac do skutku
# ValueError: base is not invertible for the given modulus

In [ ]:
print(f'd=\n{private_key[0]}\n')
print(f'n=\n{private_key[1]}\n')
print(f'private_key =\n{private_key}\n')

print(400 * '=')
print(f'\n')

print(f'e=\n{public_key[0]}\n')
print(f'n=\n{public_key[1]}\n')
print(f'private_key =\n{public_key}')

d=
11177741845808176832015209665518930031356682387397300497542422248100762976215024701331763969434027431669215157012452390687135627580668615325415001387626718873636105611540685719894235110277011084914515164972854599329081855458271963605013701924955579445645620056691642838923106478820116941221977124907198170260735653282716389113284138041661485532890153917981663767720199497356812552426376666414938607787867869885958295634386311993067143114962845701596733997283108221954343981073098895461443784690060870709176293935842572251657712582269837662744964210159008621332186366763898509822721072189775443363149713015503246020685

n=
158090643575668259446288743060005987209737488739873172230169039379315036616782700673181565561561392981884969446454102899292587336075094876150426142282915956975251636909314122809456448704809357024542590786915637395448978129660865632929895347071428124222037204477826190029722639328119014481822535722569348779486075684923590028251233393629427125603395503339063752657787222831773

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

Wiadomość musi być mniejsza lub równa liczbie $n$, w przeciwnym wypadku należy podzielić ją na bloki.

In [ ]:
def str2num(text):
  print(f'str2num::text = {text}')
  conv = text.encode('utf-8')
  number = int.from_bytes(conv, byteorder='big')
  return number

def num2str(number):
  # number.bit_length() - liczba bitów potrzebnych do zapisania liczby
  # dodanie 7 i podzielenie całości przez 8 (wynik całkowity) pozwala na zaokrąglenie w górę do pełnej liczby, co pozwala na prawidłową konwersję na tekst
  conv = number.to_bytes((number.bit_length() + 7) // 8, byteorder='big')
  text = conv.decode('utf-8')
  return text

In [ ]:
def encryptRSA(M, public_key):
  M = str2num(M)
  e, n = public_key
  C = pow(M, e, n) # nie dawać n poza modulo - liczy sie w nieskonczonosc!
  return C

def decryptRSA(C, private_key):
  d, n = private_key
  M = pow(C, d, n)
  return num2str(M)

In [ ]:
M = 'Ala ma kota'

print(str2num(M))
print(num2str(str2num(M)))

str2num::text = Ala ma kota
79091985525906541409956961
str2num::text = Ala ma kota
Ala ma kota


In [ ]:
msg_enc = encryptRSA(M, public_key)

str2num::text = Ala ma kota


In [ ]:
print(f'Encrypted M = \n{msg_enc}')

Encrypted M = 
3911609399368800141279934436914423844598508462512917730562359263965907096416681787860204435854469812483819822157808975911595022364602504693671181681956241702298305125494903579312004127636048165334685470240644052304942858471128384000009269846938979626985810754212821185819622203472112793787030483772464357137770504442263590685338540610020084221724821996574756630337453339674856923916492228646848953332618510286128616949217740622876079531626972532438807369803231395168793899392215917400652025917607952047493348048603569208707014332062213362563503849773286340599152600253050997736397992517590527220586999116192349646608


In [ ]:
msg_dec = decryptRSA(msg_enc, private_key)

In [ ]:
print(f'Decrypted M = \n{msg_dec}')

Decrypted M = 
Ala ma kota


## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [ ]:
def create_SIG(M, key):
  h = hl.sha256(M.encode('utf-8')).hexdigest()
  SIG = encryptRSA(h, key)
  return M, SIG

def verify_SIG(M, SIG, key):
  h = hl.sha256(M.encode('utf-8')).hexdigest()
  VER = decryptRSA(SIG, key)

  if VER == h:
    return True
  else:
    return False

In [ ]:
M = 'Ala ma kota'

In [ ]:
sigmsg, sig = create_SIG(M, private_key)
verify_SIG(sigmsg, sig, public_key)

str2num::text = 124bfb6284d82f3b1105f88e3e7a0ee02d0e525193413c05b75041917022cd6e


True

In [ ]:
M2 = 'Kot ma Ale'

In [ ]:
verify_SIG(M2, sig, public_key)

False

## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [ ]:
def MGF1(M, len, H=hl.sha256, hlen=256//8):
  if not isinstance(M, bytes):
    M = bytes(M, 'utf-8')

  output = bytes()

  for i in range(0, math.ceil(len / hlen)):
    tmp = H(M + i.to_bytes(4, byteorder='big')).digest() # 4 bytes = size of int
    output = output + tmp
  return output[:len]

In [ ]:
MGF1('', 4)

b'\xdf?a\x98'

In [ ]:
MGF1('', 8)

b'\xdf?a\x98\x04\xa9/\xdb'

In [ ]:
MGF1('', 16)

b'\xdf?a\x98\x04\xa9/\xdb@W\x19-\xc4=\xd7H'

In [ ]:
M = 'Ala ma kota'

In [ ]:
MGF1(M, 5)

b'O\x82y,\xd5'

In [ ]:
MGF1(M, 10)

b'O\x82y,\xd5\x98\x18\xc5B7'

In [ ]:
MGF1(M, 20)

b'O\x82y,\xd5\x98\x18\xc5B7\xc4YT\xe6\xfa\x82\xdd\x08\xc8\x8d'

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [ ]:
# def bytes_xor(n1, n2):
#   result = int.from_bytes(n1, byteorder='big') ^ int.from_bytes(n2, byteorder='big')
#   result = result.to_bytes((result.bit_length() + 7) // 8, byteorder='big')
#   return result
def bytes_xor(n, m):
  xored = int.from_bytes(n, byteorder='big') ^ int.from_bytes(m, byteorder='big')
  s = math.ceil(xored.bit_length() / 8)
  return xored.to_bytes(s, byteorder='big')

In [ ]:
def OAEP_encode(k, M, H=hl.sha256, hlen=256//8, mgf=MGF1):
  mlen = len(M)
  # print(f'mlen = {mlen}')
  if mlen > k - 2 * hlen - 2:
    raise ValueError('Algorithm pt. 1: "mlen > k - 2 * hlen - 2" criterium met (message is too long)')

  # print(f'OAEP_encode::M = {M}')
  if not isinstance(M, bytes):
    M = bytes(M, 'utf-8')

  lHash = H(b'').digest()

  PS = bytes(k - mlen - 2 * hlen - 2)
  # print(PS)

  DB = lHash + PS + b'\x01' + M
  # print(f'len(DB) = {len(DB)}')
  # print(f'(k - hlen - 1) = {(k - hlen - 1)}')

  if len(DB) != (k - hlen - 1):
    raise ValueError('Algorithm pt. 4: "len(DB) == (k - hlen - 1)" criterium not met')
  
  seed = b''.join([cu.getRandomRange(0, 256).to_bytes(1, byteorder='big') for _ in range(hlen)])
  # print(seed)

  dbMask = mgf(seed, k - hlen - 1)
  maskedDB = bytes_xor(DB, dbMask) # DB ^ dbMask
  # print(maskedDB)
  seedMask = mgf(maskedDB, hlen)
  maskedSeed = bytes_xor(seed, seedMask) # seed ^ seedMask

  EM = b'\x00' + maskedSeed + maskedDB

  if len(EM) != k:
    raise ValueError('Algorithm pt. 10: "len(EM) == k" criterium not met')
  
  return EM

def OAEP_decode(k, EM, mlen, H=hl.sha256, hlen=256//8, mgf=MGF1):
  # print(f'EM = {EM}')
  # print(f'mlen = {mlen}')
  Y = EM[0]

  # print(f'Y = \n{Y.to_bytes(1, byteorder='big')}')

  if not Y.to_bytes(1, byteorder='big') == b'\x00':
    raise ValueError('Algorithm pt. 8: "Y == b\\x00" criterium not met')

  maskedSeed = EM[1:hlen + 1] # desc tells that it should be hlen but hlen + 1 is ok
  maskedDB = EM[hlen + 1::]
  
  # print(f'maskedSeed = \n{maskedSeed}')
  # print(f'maskedDB = \n{maskedDB}')

  lHash = H(b'').digest()
  seedMask = mgf(maskedDB, hlen)
  seed = bytes_xor(maskedSeed, seedMask)
  dbMask = mgf(seed, k - hlen - 1)
  DB = bytes_xor(maskedDB, dbMask)

  # print(f'DB = \n{DB}')

  lHash_prim = DB[:hlen]
  # print(f'lHash_prim = \n{lHash_prim}')
  # print(f'lHash = \n{lHash}')

  if not lHash == lHash_prim:
    raise ValueError('Algorithm pt. 9: "lHash == lHash_prim" criterium not met')

  PS = DB[hlen:(k - mlen - hlen - 2)] # (k - mlen - hlen - 2) instead of (k - mlen - 2 * hlen - 2)
  
  # print(f'PS = \n{PS}')
  # print(f'k - mlen - 2 * hlen - 2) = \n{(k - mlen - hlen - 2)}')

  if not DB[(k - mlen - hlen - 2):(k - mlen - hlen - 2) + 1] == b'\x01':
    raise ValueError('Algorithm pt. 7: "DB[(k - mlen - 2 * hlen - 2) + 1] == b\\x01" criterium not met')
  
  M = DB[(k - mlen - hlen - 2) + 1::]

  # return M.decode('utf-8')
  return M

In [ ]:
M = 'Ala ma kota'

OAEP_encode_res = OAEP_encode(nlen, M)
print(f'OAEP encode result: \n{OAEP_encode_res}')

OAEP_decode_res = OAEP_decode(nlen, OAEP_encode_res, len(M))
print(f'OAEP decode result: \n{OAEP_decode_res.decode("utf-8")}')

OAEP encode result: 
b'\x00:\x97.\xf8cR\xad\x84\xaf\xe9\xd2\r\x8c2\xe7\xe9Af\x7f~\x07\x03\xde\n\xde\x85\xa1\xa2\x0f\x92\xf5,\xac\xd5\xb2\x92\x94\xb4M\x9f\xa34\xe0\xbcD\xac+\xba\xe65y\xdb\xd1\xad\xc1\xe7\x012\x97\xd2\xbe\xed\xdc\xe8\x981k\xca\x0e\x07][\x90.\xa3\x087N\x94Cj\xb1E\xc67rO\xac\n\xe3)\xef\x0c\xd8\xd8\xfc\x97\xe3\x0b\xc9\xcc\xb0\xb0\xf6\x07\xb6\xde\xd64X\x0b}\xe6@]7y\xca\x91\xad\xb90\x03eGno7\x1d\xfe\xab/\x141s\xefv\x8b\xae\xf6\xa4\xaf]U~\xa6p.\x1bA\xf6\xf0>\xeb\xc6W\x84\x91\xbb\x99*\x19\x08\xe5\xb3\xd9\xac\x04\xee7\x16\x89*\xff>\xac\xcf\x05s\tNZ\xff>\xa2\xc1\xc0\x16\xda\xbe\xa4f\xc1\x97g8\xde\x8d\xcc\xf7\xcao\xc6\x9a7\x92\xdc\xce\xfc\x12\xc3\x7f3\xf6\xda\xee>\x83d\x9b\xd1!\xdc\x84\xdc\x15\x80\x10\xae\xf9\xcd\x1bG\xcd\xc5*&\x87\x0e\x06R?\xd0\xa9\x99\xff\x0fj#c\xb9\xf9;\x99\xe5{\xb2#\xc3\x9asrHO^\xf4\x7fEX\xfb8\x0f\x16\xb5\xc8"\x17\xd1&\xe3\xff!\x08\xf7\xb2oc\x08P\xe8d=Q\xfb\xaf\xe7\xda\x1cf`9=n\x02\xea\xef\xf4\x97\xd0\xdf\xee\xb4\xa8+|\x15\x8blC\xe752Y\xbc\xc2\xf6L\x9b\x87o\'#

In [ ]:
def encryptRSA_OAEP(M, public_key):
  e, n = public_key
  k = n.bit_length() // 8
  M = OAEP_encode(k, M)
  M = int.from_bytes(M, byteorder='big')
  C = pow(M, e, n)
  return C

def decryptRSA_OAEP(C, mlen, private_key):
  d, n = private_key
  k = n.bit_length() // 8
  M = pow(C, d, n)
  M = M.to_bytes(k, byteorder='big')
  M = OAEP_decode(k, M, mlen)
  return M

In [ ]:
M = 'Ala ma kota'

encryptRSA_OAEP_res = encryptRSA_OAEP(M, public_key)
print(f'encryptRSA_OAEP: \n{encryptRSA_OAEP_res}')

decryptRSA_OAEP_res = decryptRSA_OAEP(encryptRSA_OAEP_res, len(M), private_key)
print(f'decryptRSA_OAEP_res: \n{decryptRSA_OAEP_res}')

encryptRSA_OAEP: 
7499928425166095434223677342457267668314097645040883345809832720529417303589550030950374212339816482169613324528298536666566350286494651525968456190476955830422008525196593741317691063271426449550479921920138057120174523049889652876570254150297131709522480048021537453811329927134847629645049284874654169792920385708115649233749431080115229455702253479876886681707972622664369622763229696503713916286829209946378127029773684843680968787504209721040980956732652309258080606208521735487271720566272210259972597537437703932238209468592936257177782970896147231802277910313260072808411625601666757156169227057520867461005
decryptRSA_OAEP_res: 
b'Ala ma kota'


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [ ]:
def EMSA_encode(emlen, M, H=hl.sha256, hlen=256//8, mgf=MGF1):
  emBits = emlen * 8 - 1
  slen = hlen
  M = bytes(M, 'utf-8')

  mHash = H(M).digest()
  if emlen < hlen + slen + 2:
    raise ValueError('Algorithm pt. 2: "emlen < hlen + slen + 2" criterium met')
  
  salt = bytearray(random.getrandbits(4) for _ in range(slen))
  # print(f'salt =\n{salt}')
  # print(f'len(salt) =\n{len(salt)}')
  
  # print(f'mHash = {mHash}')
  # print(f'len(mHash) = {len(mHash)}')
  # print(f'salt = {salt}')
  # print(f'len(salt) = {len(salt)}')

  M_prim = b'\x00\x00\x00\x00\x00\x00\x00\x00' + mHash + salt # M_prim = b'\x0000000000000000' + mHash + salt
  # print(f'len(M_prim) = {len(M_prim)}')
  # print(f'8 + hlen + slen = {8 + hlen + slen}')

  if not len(M_prim) == 8 + hlen + slen:
    raise ValueError('Algorithm pt. 4: "len(M_prim) == 8 + hlen + slen" criterium not met')
  
  mHash_prim = H(M_prim).digest()
  # print(f'EMSA_encode::mHash_prim = {mHash_prim}')
  PS = bytes(emlen - slen - hlen - 2)

  DB = PS + b'\x01' + salt

  if not len(DB) == emlen - hlen - 1:
    raise ValueError('Algorithm pt. 7: "emlen - hlen - 1" criterium not met')
  
  dbMask = mgf(mHash_prim, emlen - hlen - 1)
  maskedDB = bytes_xor(DB, dbMask)

  # print(f'EMSA_encode:: len(maskedDB)= {len(maskedDB)}')

  # bin -> format 0b -> dlatego [2:] usuwa ten prefiks
  # zfill dopełnia do pełnej długości
  binary = bin(int.from_bytes(maskedDB, byteorder='big'))[2:].zfill(len(maskedDB) * 8)
  # print(f'EMSA_encode:: binary beofre zero = \n{binary}')
  binary = '0' * (8 * emlen - emBits) + binary[8 * emlen - emBits:]
  # print(f'EMSA_encode:: binary = \n{binary}')
  # print(f'EMSA_encode:: len(binary)= {len(binary)}')

  num = int(binary, 2)
  maskedDB = num.to_bytes(len(maskedDB), byteorder='big')
  
  # print(f'maskedDB = \n{maskedDB}')
  # print(f'len(maskedDB) = {len(maskedDB)}')
  # print(f'EMSA_encode::8 * emlen - emBits = {8 * emlen - emBits}')
  # print(f'maskedDB[8 * emlen - emBits:] = \n{maskedDB[:8 * emlen - emBits]}')
  # print(f'maskedDB[8 * emlen - emBits:8 * emlen - emBits + 1] (next bit) = \n{maskedDB[8 * emlen - emBits:8 * emlen - emBits + 1]}')

  EM = maskedDB + mHash_prim + b'\xbc'

  # print(f'EMSA_encode::len(EM) = {len(EM)}')
  return EM

def EMSA_decode(emlen, EM, M, H=hl.sha256, hlen=256//8, mgf=MGF1):
  emBits = emlen * 8 - 1
  slen = hlen
  # print(f'EM = {EM}')
  # print(f'len(EM) = {len(EM)}')

  M = bytes(M, 'utf-8')
  mHash = H(M).digest()

  # print(f'mHash = {mHash}')

  if emlen < hlen + slen + 2:
    raise ValueError('Algorithm pt. 2: "emlen < hlen + slen + 2" criterium met')
  
  # print(f'EM[-1] = {EM[-1]}')
  # print(f'EM[-1].to_bytes(1, byteorder="big") = {EM[-1].to_bytes(1, byteorder="big")}')

  if EM[-1].to_bytes(1, byteorder='big') != b'\xbc':
    raise ValueError('Algorithm pt. 3: "M[-1] != b\\xbc" criterium not met')

  maskedDB = EM[:emlen - hlen - 1]
  # print(f'maskedDB = {maskedDB}')
  # print(f'len(maskedDB) = {len(maskedDB)}')
  # print(f'len(maskedDB) * 8 = {len(maskedDB) * 8}')
  mHash_prim = EM[emlen - hlen - 1:emlen - 1]
  # print(f'mHash_prim = {mHash_prim}')
  
  # print(f'EMSA_decode:: emlen - hlen - 1 = {emlen - hlen - 1}')
  # print(f'EMSA_decode::8 * emlen - emBits = {8 * emlen - emBits}')

  if not bin(int.from_bytes(maskedDB, byteorder='big'))[2:].zfill(len(maskedDB) * 8)[:8 * emlen - emBits] == '0' * (8 * emlen - emBits):
    raise ValueError('Algorithm pt. 5: "maskedDB[8 * emlen - emBits] == 0" criterium not met')

  dbMask = mgf(mHash_prim, emlen - hlen - 1)

  # print(f'dbMask = \n{dbMask}')
  # print(f'maskedDB = \n{maskedDB}')

  DB = bytes_xor(maskedDB, dbMask)
  # print(f'DB = {DB}')
  binary = bin(int.from_bytes(DB, byteorder='big'))[2:].zfill(len(DB) * 8)
  # print(f'binary before zeros = \n{binary}')
  # print(f'(8 * emlen - emBits) = {(8 * emlen - emBits)}')
  binary = '0' * (8 * emlen - emBits) + binary[8 * emlen - emBits:]
  num = int(binary, 2)
  DB = num.to_bytes(len(DB), byteorder='big')

  # print(f'binary = \n{binary}')
  # print(f'num = {num}')
  # print(f'DB = {DB}')
  # print(f'DB[:emlen - hlen - slen - 2] = {DB[:emlen - hlen - slen - 2] }')
  # tempvar = b'\x00' * (emlen - hlen - slen - 2)
  # print(f'b"\\x00" * (emlen - hlen - slen - 2) = {tempvar}')

  if DB[:emlen - hlen - slen - 2] != b'\x00' * (emlen - hlen - slen - 2):
    raise ValueError('Algorithm pt. 9: "DB[emlen - hlen - slen - 2] == 0" criterium not met')
  
  # print(f'len(DB) = {len(DB)}')
  # print(f'DB[emlen - hlen - slen - 2:emlen - hlen - slen - 1] = {DB[emlen - hlen - slen - 2:emlen - hlen - slen - 1]}')
  if DB[emlen - hlen - slen - 2:emlen - hlen - slen - 1] != b'\x01':
    raise ValueError('Algorithm pt. 9: "DB[emlen - hlen - slen - 1] == b\\x01" criterium not met')

  salt = DB[-slen:]
  M_prim = b'\x00\x00\x00\x00\x00\x00\x00\x00' + mHash + salt
  if not len(M_prim) == 8 + hlen + slen:
    raise ValueError('Algorithm pt. 11: "len(M_prim) == 8 + hlen + slen" criterium not met')
  
  mHash_bis = H(M_prim).digest()

  if not mHash_prim == mHash_bis:
    raise ValueError('Algorithm pt. 13: "mHash_prim == mHash_bis" criterium not met')
  
  return True

In [ ]:
M = 'Ala ma podpis'

EMSA_encode_res = EMSA_encode(nlen//8, M)

print(f'M = \n{M}')
print(f'EMSA_encode_res = \n{EMSA_encode_res}\n')
print(f'EMSA_decode verify: \n{EMSA_decode(nlen//8, EMSA_encode_res, M)}')


M = 
Ala ma podpis
EMSA_encode_res = 
b'\x1f>\x9e\xa2\xd7\xff\x98N\x1a\xbd\xb0\xc7<\x91BY\x10\xb6\x93\x12\x1e\n\x1d\xef8\x06\x13\xb3Z\x08\xf8>+*\xc8\xb5\xbe\xbd\xd2\x8b&\x8an\xac\x1e\xe7 }1\xd9^\x7f\xa2\xf3\x84x\x96\nY\n\xb9\x8d\x06l\xbarF\x7fH\x83\x8dw\xde\x80$\xd3\x85\xd1\x08\x93*LE\xde\xfbW\xe3a\n+\xa5\xf9\xb1y\xb8\xe1k\xd7\xd6Q\xa8\xc2W\x16Y\xc1\x1c\x1f#!o\x84\x9d\x87\x14{\x9a\x18\x89\xa4\xe7)\xe8|\xd9\xe7\x07\xc4\x19ld\xa3\xfc\xd7,\xd6\x16\xae\xb31\xde\x9dv|x\xd87\xddk?+\xd7\xef\x8a\x9fFE\xd4\xa0\xa1?]p\xcf\x91\n\xb2 \x17\xad\x13L\xf5\x8e;F\xd4\xf3c\x86\x91\x11\r\xe8\xed\x85m_~J\xe7?"@\xef!I\x18\xd3\x8b\xd4TkY\xe4&\xf1,\xef\xb6^\xb0&UV\x0c\x00lL\xab\xa7\xaa\xac\r\xc4\xbc\x94\xa9\x13\xbeQ\xee\xa5\xf6?\x98\x02\xa3\xff\xe8\x93\xf5\xb4:\xf9\x8f\xf4F\xf8\x96\xb1kGHM\xbc'

EMSA_decode verify: 
True


In [ ]:
def create_SIG_EMSA(M, key, nlen=2048//8):
  EMSA_encode_res = EMSA_encode(nlen, M)
  SIG = pow(int.from_bytes(EMSA_encode_res, byteorder='big'), key[0], key[1])
  return M, SIG

def verify_SIG_EMSA(M, SIG, key, nlen=2048//8):
  VER = pow(SIG, key[0], key[1])

  try:
    return EMSA_decode(nlen, VER.to_bytes(nlen, byteorder='big'), M)
  except ValueError:
    return False

In [ ]:
M = 'Ala ma podpis'
M2 = 'Podpis ma Ale'

create_SIG_EMSA_result = create_SIG_EMSA(M, private_key)
print(f'Message 1: {M}')
print(f'Message 2: {M}')
print(f'Verify result: {verify_SIG_EMSA(M, create_SIG_EMSA_result[1], public_key)}')

Message 1: Ala ma podpis
Message 2: Ala ma podpis
Verify result: True


In [ ]:
print(f'Message 1: {M}')
print(f'Message 2: {M2}')
create_SIG_EMSA_result = create_SIG_EMSA(M, private_key)
print(f'Verify result: {verify_SIG_EMSA(M2, create_SIG_EMSA_result[1], public_key)}')

Message 1: Ala ma podpis
Message 2: Podpis ma Ale
Verify result: False


## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [ ]:
def encrypt_key_RSA_KEM(pkey, H=hl.sha256):
  rand = random.randint(0, 256).to_bytes(1, byteorder='big')
  key = H(rand).digest()
  cipherered_key = encryptRSA_OAEP(key, pkey)
  return cipherered_key, key

def decrypt_key_RSA_KEM(cipherered_key, key, pkey):
  # print(f'cipherered_key = {cipherered_key}')
  # print(f'key = {key}')
  return decryptRSA_OAEP(cipherered_key, len(key), pkey)

In [ ]:
encrypt_key_RSA_KEM_res = encrypt_key_RSA_KEM(public_key)
decrypt_key_RSA_KEM_res = decrypt_key_RSA_KEM(encrypt_key_RSA_KEM_res[0], encrypt_key_RSA_KEM_res[1], private_key)

print(f'encrypt_key_RSA_KEM key = \n{encrypt_key_RSA_KEM_res[1]}')
print(f'decrypt_key_RSA_KEM key = \n{decrypt_key_RSA_KEM_res}\n')

print(f'Is equal?:')
if decrypt_key_RSA_KEM_res == encrypt_key_RSA_KEM_res[1]:
  print(f'True')
else:
  print(f'False')

encrypt_key_RSA_KEM key = 
b'\t\xfc\x96\x08-4\xc2\xdf\xc1)]\x92\x07;^\xa1\xdc\x8e\xf8\xda\x95\xf1M\xfd\xed\x01\x1f\xfb\x96\xd3\xe5K'
decrypt_key_RSA_KEM key = 
b'\t\xfc\x96\x08-4\xc2\xdf\xc1)]\x92\x07;^\xa1\xdc\x8e\xf8\xda\x95\xf1M\xfd\xed\x01\x1f\xfb\x96\xd3\xe5K'

Is equal?:
True
